In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101
from random_search import run_random_search, random_spec
from neural_predictor import classifier, regressor
from input_preprocessing import preprocess_nasbench

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 44 seconds


### Two-stage vs One-stage prediction 10 split (N=172)

In [4]:
N = 172

In [5]:
def get_N_samples(N):
    models = []

    for _ in range(N):
        while True:
            model = random_spec(nasbench)
            if model not in models:
                models.append(nasbench.query(model))
                break
                
    return preprocess_nasbench(models)

In [6]:
LOOPS = 60
MAX_SAMPLES = 500
MAX_TIME_BUDGET = 8e5

In [7]:
train_data = get_N_samples(N)
reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_acc'])

In [8]:
train_time = np.sum(train_data['times'])

### One-stage

In [9]:
np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for budget in tqdm(range(int(train_time), int(MAX_TIME_BUDGET), 1600)): # 5000 loops
        
    loop_val, loop_test = [], []
    for _ in range(LOOPS):
        time_spent = 0
        test_models = get_N_samples(N+budget//100)
        
        for i in range(len(test_models['times'])):
            time_spent = time_spent + test_models['times'][i]
            if time_spent >= budget:
                break
        
        X = test_models['X'][:i]
        A = test_models['norm_A'][:i]
        AT = test_models['norm_AT'][:i]
        labels = test_models['val_acc'][:i]
        test_labels = test_models['test_acc'][:i]
        
        pred_acc = reg.predict([X, A, AT]).ravel()
        
        selected_val = labels
        selected_test = test_labels
        
        best_val_idx = np.argmax(selected_val)
        
        loop_val.append(selected_val[best_val_idx])
        loop_test.append(selected_test[best_val_idx])

    val_acc.append(np.mean(loop_val))
    test_acc.append(np.mean(loop_test))
    
    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(loop_val))
    
    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(loop_test))

  0%|          | 0/367 [00:00<?, ?it/s]

In [10]:
np.save('outputs/np_val_avg_by_time.npy', np_val_avg)
np.save('outputs/np_val_std_by_time.npy', np_val_std)
np.save('outputs/np_test_avg_by_time.npy', np_test_avg)
np.save('outputs/np_test_std_by_time.npy', np_test_std)